In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
!pip install np_utils

In [4]:
import numpy as np
import pandas as pd
from plotly.offline import iplot
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
from tensorflow.keras.metrics import Recall
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))
from tqdm import tqdm

2023-04-24 08:07:57.849334: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-24 08:08:00.240241: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-24 08:08:00.241063: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-24 08:08:02.115049: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
politifact = pd.read_json(r'./data/politifact.json')
snopes = pd.read_json(r'./data/snopes.json')

In [6]:
# column names
poli_cols = politifact.columns.to_list()
snop_cols = snopes.columns.to_list()

poli_row_count, poli_col_count = politifact.shape
snop_row_count, snop_col_count = snopes.shape

In [7]:
common_cols = set(poli_cols) & set(snop_cols)

td = pd.concat([snopes[list(common_cols)], politifact[list(common_cols)]])
td = td.reindex(columns = ['doc', 'claim', 'factchecker', 'url', 'sources', 'topic', 'published' ,'label' ])


In [8]:
td2 =td.dropna(subset=['doc','claim','label'])

In [9]:
print(td2.shape, td.shape)

(22298, 8) (22298, 8)


In [10]:
td2['label']=td2.label.map(lambda x: x.lower())

In [11]:
valid_labels = ['false','true','half-true','mostly-true','barely-true','pants-fire','mixture']

td2 = td2[td2.label.isin(valid_labels)]

In [12]:
td2 = td2.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):

    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
td2['claim'] = td2['claim'].apply(clean_text)
td2['doc'] = td2['doc'].apply(clean_text)

In [13]:
td2['body'] = td2['doc'].str.cat(td2['claim'], sep=' ')

In [14]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(td2['body'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 223445 unique tokens.


In [15]:
X = tokenizer.texts_to_sequences(td2['body'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (20874, 250)


In [16]:
Y = pd.get_dummies(td2['label']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (20874, 7)


In [17]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(18786, 250) (18786, 7)
(2088, 250) (2088, 7)


In [18]:
EMBEDDING_DIM

100

In [19]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(7, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 250, 100)          5000000   
                                                                 
 spatial_dropout1d (SpatialD  (None, 250, 100)         0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 7)                 707       
                                                                 
Total params: 5,081,107
Trainable params: 5,081,107
Non-trainable params: 0
_________________________________________________________________
None


In [20]:
epochs = 10
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])
     

Epoch 1/10
265/265 [==============================] - 56s 202ms/step - loss: 1.0361 - accuracy: 0.6204 - val_loss: 0.3630 - val_accuracy: 0.8962
Epoch 2/10
265/265 [==============================] - 54s 204ms/step - loss: 0.2210 - accuracy: 0.9299 - val_loss: 0.2131 - val_accuracy: 0.9202
Epoch 3/10
265/265 [==============================] - 54s 204ms/step - loss: 0.1177 - accuracy: 0.9648 - val_loss: 0.2536 - val_accuracy: 0.9090
Epoch 4/10
265/265 [==============================] - 54s 203ms/step - loss: 0.0589 - accuracy: 0.9854 - val_loss: 0.2732 - val_accuracy: 0.9234
Epoch 5/10
265/265 [==============================] - 54s 206ms/step - loss: 0.0345 - accuracy: 0.9909 - val_loss: 0.2868 - val_accuracy: 0.9260


In [29]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(accuracy,loss)

66/66 [==============================] - 2s 27ms/step - loss: 0.2959 - accuracy: 0.9296
0.9295976758003235 0.2959437370300293


In [1]:
import matplotlib.pyplot as plt

# get the accuracy and loss values from the history object
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

# plot the accuracy and loss curves
plt.plot(accuracy, label='Training Accuracy')
plt.plot(val_accuracy, label='Validation Accuracy')
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend()
plt.show()
